In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Membuat dataset dummy
np.random.seed(42)
data_dummy = {
    "Time": np.random.randint(0, 172800, size=1000),  # Waktu dalam detik
    "V1": np.random.randn(1000),
    "V2": np.random.randn(1000),
    "V3": np.random.randn(1000),
    "V4": np.random.randn(1000),
    "Amount": np.random.uniform(1, 500, size=1000),  # Jumlah transaksi
    "Class": np.random.choice([0, 1], size=1000, p=[0.99, 0.01])  # 1% fraud
}

# Konversi ke DataFrame
data = pd.DataFrame(data_dummy)

In [4]:
print(data.head())

     Time        V1        V2        V3        V4      Amount  Class
0  121958 -2.289061 -1.313758 -0.452562 -0.392802   64.778110      0
1  146867  1.432482 -1.095302 -0.129910 -1.362911  291.536487      0
2  131932  1.214722 -0.168797  0.581433  0.699020  150.637260      0
3  103694 -0.880864  0.110956 -0.203236 -0.243510   18.167201      0
4  119879 -0.881879  0.786970  1.110118  0.015365  317.829935      0


In [5]:
# 2. Menyiapkan Fitur dan Label
X = data.drop("Class", axis=1)  # Semua kolom kecuali 'Class'
y = data["Class"]               # Kolom 'Class' sebagai label (0: normal, 1: fraud)

# 3. Normalisasi Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Membagi Dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Membangun Model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 6. Melatih Model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2
)

# 7. Evaluasi Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 8. Prediksi dan Analisis
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 9. Menyimpan Model
model.save("fraud_detection_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.2721 - loss: 0.7921 - val_accuracy: 0.5437 - val_loss: 0.6705
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6921 - loss: 0.6352 - val_accuracy: 0.9250 - val_loss: 0.5446
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8765 - loss: 0.5281 - val_accuracy: 0.9688 - val_loss: 0.4418
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9576 - loss: 0.4293 - val_accuracy: 0.9937 - val_loss: 0.3553
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9918 - loss: 0.3511 - val_accuracy: 0.9937 - val_loss: 0.2819
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9899 - loss: 0.2880 - val_accuracy: 0.9937 - val_loss: 0.2220
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9895 - loss: 0.2241 - val_accuracy: 0.9937 - val_loss: 0.1736
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9844 - loss: 0.1959 - val_accuracy: 0.9937 - v

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[[198   0]
 [  2   0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       198
           1       0.00      0.00      0.00         2

    accuracy                           0.99       200
   macro avg       0.49      0.50      0.50       200
weighted avg       0.98      0.99      0.99       200



In [8]:
# Load model
model = tf.keras.models.load_model("fraud_detection_model.h5")

# Use 'data' instead of 'df' for prediction data
X_new = data.drop("Class", axis=1) # Changed 'df' to 'data'

# Normalisasi
scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new)

# Prediksi
predictions = (model.predict(X_new_scaled) > 0.5).astype("int32")
predictions

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    